In [4]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
import pprint
import s3fs

import models, cross_validation, comparator

In [6]:
path = "s3://aigluence-data2/data/02_intermediate/chatelet_pdt/H.parq"
df = pd.read_parquet(path)

# on a un csv avec multiindex "line", "station", "datetime", "direction"

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
models_to_test = [
  models.ImputeByMean(["datetime.dt.hour", "datetime.dt.dayofweek", "direction"]), 
  models.ImputeByMedian(["datetime.dt.hour", "datetime.dt.dayofweek", "direction"]), 
  models.RandomImpute(),
  models.ImputeLOCF(["direction"]),
  models.ImputeNOCB(["direction"]),
  models.ImputeKNN(),
#   models.ImputeProphet(**{
#               "weekly_seasonality": True, 
#               "yearly_seasonality": True,
#               "interval_width": 0.95,
#               })
# models.ImputeRPCA(),
]

search_params = {
  "ImputeKNN": {"k": {"min":2, "max":3, "type":"Integer"}},
  "ImputeProphet": {
          "daily_seasonality": {"categories": [True, False], "type":"Categorical"},
        }
  }

line = "H"
station = "VALMONDOIS"
dataset = df[(df["line"]==line) & (df["station"]==station)]
cols_to_impute = ["load"]

dataset = dataset.set_index(["line","station", "datetime", "direction"])
# dataset = dataframe with multiindex and at least, columns in cols_to_impute

prop_nan = 0.1


comparison = comparator.Comparator(
          dataset,
          prop_nan, 
          models_to_test, 
          cols_to_impute,
          search_params=search_params,
)
results = comparison.compare()

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(results)